In [ ]:
import pandas as pd

In [ ]:
#!find . -maxdepth 3 -type f -exec du -h {} + | sort --human-numeric-sort --reverse
!free

## INTEGRATE

In [3]:
url= "https://docs.google.com/spreadsheets/d/e/2PACX-1vSXaZ3xwCNvUTs3DgqzCaE3W481KJQhC6FqbNQ6zU04AGCj37vqS5TEMidyh46NC6UlyCbSXtj52UOc/pub?output=csv"
source = "psych1k_100522"

In [4]:
df = pd.read_csv(url)

In [5]:
df["labels"] = df["label"]
df["source"] = source
df = df.drop(columns=["predicts"])

In [6]:
df = df.dropna(subset=["labels"])
df["labels"] = df["labels"].astype(int) 
df = df[(df["labels"] == 1) | (df["labels"] == 0)] #fix typos in labels eg. misstyped 0 as 9


In [10]:
df2 = pd.read_csv("downloads/43k_pm_bio.csv")
df3 = df2.append(df)
df3.to_csv("downloads/47k_train.csv", index = False, header=True)

## TRAIN